# E-charging Infrastructure in Germany

In this article we are going to perform an analysis of the charging infrastracture in Germany.It is like a follow up of an article I stumbled upon a while back, by Amanda Iglesias Moreno and can be found [here](https://towardsdatascience.com/e-mobility-in-germany-analysis-of-electric-vehicle-charging-stations-58d797988738). I however give it a different twist, by answering the various questions using SQL. This is more like ‘translating pandas syntax to SQL’, if you compare the results from my analysis and Amandas’s side by side. The goal is to show the intersection between SQL and Python in analysis. <br> <br>A quick mention is that dataset contents have changed over time and so some of the steps in data cleaning are different. This is an important skill for a data analyst as in many situations in practice, one has to contribute further to existing projects.

With that let's dive in, the main steps are:
<ol>
<li>Reading the data</li>
<li>Exploratory data analysis and cleaning</li>
<li>Connecting to database and querying</li>
<li>Conclusions and recommendations</li>
    
<ul>


### 1. Reading the data
We read the dataset directly from the website and important is including the ‘iso-8859-1’ encoding to cater for German umlauts in the dataset. We also skip the first 10 rows as they contain descriptive information about the data.

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from deep_translator import GoogleTranslator

In [2]:
# Link to dataset
url = 'https://www.bundesnetzagentur.de/SharedDocs/Downloads/DE/Sachgebiete/Energie/Unternehmen_Institutionen/E_Mobilitaet/Ladesaeulenregister_CSV.csv?__blob=publicationFile&v=42'

# Load data to a pandas dataframe
df = pd.read_csv(url, encoding='iso-8859-1', sep=';', skiprows=10)

# Visualize top two records
df.head(2)

,Betreiber,Straße,Hausnummer,Adresszusatz,Postleitzahl,Ort,Bundesland,Kreis/kreisfreie Stadt,Breitengrad,Längengrad,...,Public Key1,Steckertypen2,P2 [kW],Public Key2,Steckertypen3,P3 [kW],Public Key3,Steckertypen4,P4 [kW],Public Key4
0,Albwerk GmbH & Co. KG,Ennabeurer Weg,0,NaN,72535,Heroldstatt,Baden-Württemberg,Landkreis Alb-Donau-Kreis,"48,442398","9,659075",...,NaN,AC Steckdose Typ 2,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,smopi®,Albstraße,14,NaN,72535,Heroldstatt,Baden-Württemberg,Landkreis Alb-Donau-Kreis,"48,449353","9,672201",...,NaN,AC Steckdose Typ 2,22,NaN,AC Steckdose Typ 2,22,NaN,AC Steckdose Typ 2,22,NaN


## 2. Exploratory data analysis and cleaning

Since the dataset is in German, we translate the column headers and a few of the descriptive contents to English as we go along. For this I used the deep_translator API, a free tool to translate between different languages using multiple translators. The google translator as we will see later is less intuitive compared to deepl, but I chose it as it does not require an API key.<br><br>
The headers have spaces which is not good practice and so we replace them with hyphens

In [3]:
# Deep_translator syntax
GoogleTranslator(source='auto', target='en').translate("Guten morgen") 

'Good morning'

In [4]:
# Translate column headers to English
df.rename(columns=lambda x: GoogleTranslator(source='auto', target='en').translate(x), inplace=True)

# Replace spaces with hyphens
df.rename(columns=lambda x: x.replace(" ","_"), inplace=True)

# View top 2 rows after translation
df.head(2)

,operator,Street,House_number,address_supplement,Postal_code,Ort,Federal_State,District/district-free_city,latitude,longitude,...,Public_Key1,Connector_Types2,P2_[kW],Public_Key2,Connector_Types3,P3_[kW],Public_Key3,Connector_Types4,P4_[kW],Public_Key4
0,Albwerk GmbH & Co. KG,Ennabeurer Weg,0,NaN,72535,Heroldstatt,Baden-Württemberg,Landkreis Alb-Donau-Kreis,"48,442398","9,659075",...,NaN,AC Steckdose Typ 2,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,smopi®,Albstraße,14,NaN,72535,Heroldstatt,Baden-Württemberg,Landkreis Alb-Donau-Kreis,"48,449353","9,672201",...,NaN,AC Steckdose Typ 2,22,NaN,AC Steckdose Typ 2,22,NaN,AC Steckdose Typ 2,22,NaN


We use the pandas method **info()** to obtain a summary of the data.

In [5]:
# Check data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40674 entries, 0 to 40673
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   operator                     40674 non-null  object
 1   Street                       40674 non-null  object
 2   House_number                 40674 non-null  object
 3   address_supplement           5761 non-null   object
 4   Postal_code                  40674 non-null  int64 
 5   Ort                          40674 non-null  object
 6   Federal_State                40674 non-null  object
 7   District/district-free_city  40674 non-null  object
 8   latitude                     40674 non-null  object
 9   longitude                    40674 non-null  object
 10  commissioning_date           40674 non-null  object
 11  connected_load               40674 non-null  object
 12  Type_of_charging_device      40673 non-null  object
 13  number_of_charging_points    40

We use the pandas method ***info*** to obtain a summary of the data. As mentioned before, GoogleTranslator translated **‘Ort’** incorrectly, so we manually changed it to **‘Place’**.<br><br>
Our next step would be to translate the column **Type_of_charging_device** which seems to have a null value. Worth mentioning at this point is that a charging station  is considered **Fast** if it has at least one charging point with a power output larger than 22kW otherwise it is **Normal**. So we perform the translation then check the power of the station which has the  null value and assign its respective label.

In [6]:
# Replace previous wrong translation
df.rename(columns={'road':'street','Ort':'Place'},inplace =True)

# Replace existing names with translations
df.Type_of_charging_device.replace({'Normalladeeinrichtung':'Normal','Schnellladeeinrichtung':'Fast'} ,inplace=True)

# Replace operator value Privatperson with Private_person
df.operator.replace(dict.fromkeys(['Privatperson'],'Private_person'),inplace=True)

# Return the index of the values with null
null_values = df.Type_of_charging_device[df.Type_of_charging_device.isna()].index

# View the power
print(df['P1_[kW]'].iloc[null_values])

627    180
Name: P1_[kW], dtype: object


In [7]:
# Replace null value with 'Fast'
df.Type_of_charging_device.replace([np.nan],['Fast'], inplace = True)

# Confirm there are only two values as expected
print(df.Type_of_charging_device.unique())

['Normal' 'Fast']


We are doing pretty good but there is a record in the column **Federal_State** without a label which we will fill manually. The **address_supplement** column has many null values but it stays as is because many records do not have a supplement address. The **public keys** are numbers allowing users to remotely check measured values and do not are not important for our analysis and so we drop them.

In [8]:
# Replace null values by 0
columns_na = ['Connector_Types1', 'P1_[kW]', 'Connector_Types2', 'P2_[kW]',
               'Connector_Types3', 'P3_[kW]', 'Connector_Types4', 'P4_[kW]']

for column in columns_na:
    df[column] = df[column].fillna(value='0')
    
# Drop public keys
df.drop(columns=['Public_Key1', 'Public_Key2', 'Public_Key3', 'Public_Key4'], inplace=True)

# View null values
df.isnull().sum()

operator                           0
Street                             0
House_number                       0
address_supplement             34913
Postal_code                        0
Place                              0
Federal_State                      0
District/district-free_city        0
latitude                           0
longitude                          0
commissioning_date                 0
connected_load                     0
Type_of_charging_device            0
number_of_charging_points          0
Connector_Types1                   0
P1_[kW]                            0
Connector_Types2                   0
P2_[kW]                            0
Connector_Types3                   0
P3_[kW]                            0
Connector_Types4                   0
P4_[kW]                            0
dtype: int64

Since the dataset is in german the numbers separator for **Longitude**, **Latitude** and several other floats, is a comma as opposed to dot. I replaced them rapidly, although I encountered multiple errors because of spaces and special characters in the numbers in which case I handled them individually. Remember addresses remain as objects as we don't perform calculations with them. We also convert the **commisioning_date** to datetime and specify the format explicitly.

In [9]:
# One latitude has a dot at the end, remove it
df['latitude'] = df.latitude.apply(lambda x: str(x).rstrip('.'))

# P2_[kW] has a forward slash,split on it and keep the first value
df['P2_[kW]'] = df['P2_[kW]'].apply(lambda x: str(x).split('/', 1)[0])

# P3_[kW] has a forward slash, split on it and keep the first value
df['P3_[kW]'] = df['P3_[kW]'].apply(lambda x: str(x).split('/', 1)[0])

# P4_[kW] has a value with multiple spaces, fill with 0
df.loc[df['P4_[kW]'].str.startswith(' '), 'P4_[kW]'] = 0

# Replace comma with dot and parse the data type to float
type_replace = ['latitude','longitude', 'connected_load', 'P1_[kW]','P2_[kW]','P3_[kW]','P4_[kW]']

for col in type_replace:
    df[col] = df[col].str.replace(',','.').astype('float')
    
# Parse commisioning date to datetime
df['commissioning_date'] = pd.to_datetime(df['commissioning_date'], format='%d.%m.%Y')

Let's remove leading and lagging spaces for all columns with string data types. This is because pandas will treat any value with different characters including spaces as unique, which is not the behaviour we want especially when it comes to when we want to group the data. The column **Federal_state** had a null value which after abit of research, I found out that it is in **Mecklenburg-Vorpommern** state, and so we assign it manually.

In [10]:
# Select columns with object data type
string_data = df.select_dtypes(include='object').columns

# Remove leading and lagging spaces
for column in string_data:
    df[column] = df[column].str.strip()
    
# Fill null Federal_state value with its respective value 
df.loc[df['Federal_State'].isna(), 'Federal_State'] = 'Mecklenburg-Vorpommern'
    
# Confirm there are no spaces, leading or lagging white spaces in 'Place'
(df.Place.str.startswith(' ') | df.Place.str.endswith(' ')).any()

False

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40674 entries, 0 to 40673
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   operator                     40674 non-null  object        
 1   Street                       40674 non-null  object        
 2   House_number                 40674 non-null  object        
 3   address_supplement           5761 non-null   object        
 4   Postal_code                  40674 non-null  int64         
 5   Place                        40674 non-null  object        
 6   Federal_State                40674 non-null  object        
 7   District/district-free_city  40674 non-null  object        
 8   latitude                     40674 non-null  float64       
 9   longitude                    40674 non-null  float64       
 10  commissioning_date           40674 non-null  datetime64[ns]
 11  connected_load               40674 non-nu


And now to the last and probably hardest part of cleaning the dataset; Germany has cities such as **Frankfurt** and **Frankfurt (Oder)** which are two different places. In some cases in the dataset Frankfurt is  recorded as Frankfurt am main. In this case, we search for both and reassign them to Frankfurt. Another problem is that some major cities like **Berlin** have the district name in their names. This is quite difficult to do for all cities so to clean only for the top 15 cities.

Here are the steps we take to do the clean up:
<ol>
    <li>Scrap the top 15 cities in Germany and save them in  dataframe <strong>(df_top_cities)</strong></li>
<li>Create a nested list of the top cities and their duplicates (Lists are not the best suitable data structure for such a task but we use the anyway for simplicity)</li>
<li>Replace city names with districts in their  name with their respective city name apart from Frankfurt and Essen whose names are ambiguous.</li>
<li>Replace city names Frankfurt and Essen manually.</li></ol>

In [12]:
# Check for duplicates for Berlin
df.Place[df.Place.str.contains('Berlin')].unique()

array(['Berlin', 'Berlin-Pankow', 'Berlin (Tempelhof)',
       'Berlin Osterpreussen', 'Berlin-Lichterfelde', 'Berlin-Köpenick',
       'Berlin-Marzahn-Hellersdorf', 'Berlin Marzahn',
       'Berlin-Friedrichsfelde', 'Berlin-Reinickendorf', 'Berlin-Spandau',
       'Neuenhagen bei Berlin', 'Berlin Hönow', 'Rüdersdorf bei Berlin'],
      dtype=object)

#### i) Scrap the top 15 cities in Germany

In [13]:
# Scrap a table containing German cities
html_data = requests.get('http://www.citymayors.com/gratis/german_topcities.html').text

# Extract the relevant tables
soup = BeautifulSoup(html_data, "html.parser")
data = pd.read_html (str(soup))
df_cities = data[1]

# Assign first row to column names
df_cities.columns = df_cities.iloc[0]

# Top 15 cities
df_top_cities = df_cities.iloc[1:16,:]
df_top_cities.head(2)

,Rank,City,State,Population
1,1,Berlin,Berlin,3275000
2,2,Hamburg,Hamburg,1686100


#### ii) Creating a nested list of the top cities and their duplicates

In [14]:
# Create a nested list of top cities and their duplicates
city_duplicates = []
for city in df_top_cities.City:
    city_duplicates.append(list((city,df.Place[df.Place.str.contains(city)].unique())))

#Display the second city(index 1)
city_duplicates[1]

['Hamburg',
 array(['Hamburg', 'Hamburg-Wilhelmsburg', 'Hamburg-Wandsbeck',
        'Hamburg-Duvenstedt', 'Hamburg-Eidelstedt', 'Hamburg/ Moorfleet',
        'Wentorf bei Hamburg'], dtype=object)]

#### iii) Replace city names with districts in their name with their respective city name

In [15]:
# Replace the city regions with the name of main city; exclude Frankfurt and Essen
for l in city_duplicates:
    if l[0] not in ['Frankfurt', 'Essen']:
        df.replace(dict.fromkeys(list(l[1]),l[0]),inplace=True)

#### iv) Replace city names Frankfurt and Essen manually

In [16]:
# Unique values with frankfurt in their names
df.Place[df.Place.str.contains('Frankfurt')].unique()

array(['Frankfurt (Oder)', 'Frankfurt/Oder', 'Frankfurt',
       'Frankfurt am Main', 'Frankfurt-Nieder-Eschbach',
       'Frankfurt-Niederrad'], dtype=object)

In [17]:
# Check for places containing Essen in their name
df.Place[df.Place.str.contains('Essen')].unique()

array(['Essenbach', 'Essenbach/Oberahrain', 'Essen', 'Essen (Oldenburg)',
       'Addrup-Essen/Oldb.', 'Bad Essen', 'Essen-Kettwig',
       'Essen-Altenessen', 'Essenheim'], dtype=object)

In [18]:
# Replace relevant duplicates containing frankfurt
df.Place.replace(dict.fromkeys(['Frankfurt','Frankfurt am Main', 
                    'Frankfurt-Nieder-Eschbach','Frankfurt-Niederrad'], 'Frankfurt'),inplace=True)

# Replace reelvant duplicates containing EsSen in them
df.Place.replace(dict.fromkeys(['Essen-Kettwig','Essen'], 'Essen'),inplace=True)

Hurray! Data cleaning is finished and we can now proceed to the analysis part. 

## 3. Connecting to database and querying

Since our goal is to write SQL queries against our data, we import the sqlite3 module, enabling integration of the sqlite database with python. This will allow us to run queries on the jupyter environment. We start by setting up a connection to a database, and use the pandas method **‘to_sql’** to write to the database. For convenience I defined a function **run**, which takes our query and using the method **‘read_sql’** returns a dataframe.

In [19]:
import sqlite3

conn = sqlite3.connect('echarging.db')
df.to_sql('echarging', conn , if_exists='replace' , index = False)

# Function named run to execute my sql queries
def run(my_sql_query):
  return pd.read_sql(my_sql_query,conn)

With SQL the result is a table of results which, we stick to  for now as it is enough to answer our questions.

**1. How many stations are there per Federal State**

This syntax is quite common when the results of a group by are required. We select the field we are grouping by and the aggregating function which is **'count'** in this case.

In [36]:
run('''select latitude from echarging

''')

,latitude
0,48.442398
1,48.449353
2,48.510500
3,48.578534
4,48.179959
...,...
40669,51.021359
40670,51.024391
40671,51.060770
40672,51.092527


In [20]:
run(''' SELECT Federal_State, COUNT(*) AS charging_stations
        FROM echarging
        GROUP BY Federal_State
        ORDER BY charging_stations DESC ''')

,Federal_State,charging_stations
0,Bayern,8892
1,Nordrhein-Westfalen,7394
2,Baden-Württemberg,7120
3,Niedersachsen,4222
4,Hessen,3011
5,Sachsen,1642
6,Schleswig-Holstein,1567
7,Rheinland-Pfalz,1487
8,Berlin,1157
9,Hamburg,993


**2. How many stations are there per city (for Top cities)**

In [24]:
run('''SELECT Place, COUNT(commissioning_date) charging_stations
        FROM echarging
        GROUP BY Place
        ORDER BY charging_stations desc
        LIMIT 10''')

,Place,charging_stations
0,Berlin,1084
1,Hamburg,992
2,München,866
3,Stuttgart,642
4,Wolfsburg,554
5,Hannover,490
6,Düsseldorf,478
7,Ingolstadt,436
8,Rüsselsheim am Main,413
9,Dortmund,382


 **3. How many charging points at a station are there for each 'number_of_charging_points'**

In [47]:
run('''SELECT number_of_charging_points, COUNT(*) AS count_points_station
        FROM echarging
        GROUP BY number_of_charging_points
        ORDER BY count_points_station DESC''')

,number_of_charging_points,count_points_station
0,2,28958
1,1,6118
2,4,1154
3,3,1024


 **4.  How many charging point are there per city (Top cities)**

In [48]:
run('''SELECT Place, SUM(number_of_charging_points) AS number_of_points
        FROM echarging
        GROUP BY Place
        ORDER BY number_of_points DESC
        LIMIT 10''')

,Place,number_of_points
0,Berlin,2033
1,Hamburg,1985
2,München,1656
3,Stuttgart,1277
4,Wolfsburg,1031
5,Hannover,948
6,Düsseldorf,881
7,Rüsselsheim am Main,826
8,Ingolstadt,635
9,Köln,577


 **5. How many stations were installed per year**

In [27]:
run('''SELECT strftime('%Y', commissioning_date) year,COUNT(commissioning_date) AS number_of_stations
        FROM echarging
        GROUP BY year
        ORDER BY commissioning_date DESC
        LIMIT 10''')

,year,number_of_stations
0,2022,8247
1,2021,8602
2,2020,6035
3,2019,5586
4,2018,4128
5,2017,2002
6,2016,1159
7,2015,559
8,2014,286
9,2013,127


**6. What's the number of the different Powers of charging points (common ones)**

In [28]:
run('''
--Concatenate all the points excluding values of zero
    WITH charging_points AS(
            SELECT "P1_[kW]" AS points
            FROM echarging
            WHERE "P1_[kW]" !=0
        UNION ALL
            SELECT "P2_[kW]" AS points
            FROM echarging
            WHERE "P2_[kW]" !=0
        UNION ALL
            SELECT "P3_[kW]" AS points
            FROM echarging
            WHERE "P3_[kW]" !=0
        UNION ALL
            SELECT "P4_[kW]" AS points
            FROM echarging
            WHERE "P4_[kW]" !=0)

--select the required parameters
SELECT points as common_points,
       COUNT(points) AS number_of_points
FROM charging_points
GROUP BY points
ORDER BY number_of_points DESC
LIMIT 10
''')

,common_points,number_of_points
0,22.0,48352
1,11.0,10789
2,50.0,2536
3,150.0,2521
4,300.0,2404
5,43.0,1212
6,3.7,550
7,350.0,482
8,75.0,442
9,250.0,348


 **7. What is the maximum Power of charging points**

In [49]:
run('''
WITH power AS(
            SELECT "P1_[kW]" AS points
            FROM echarging
            WHERE "P1_[kW]" !=0
        UNION ALL
            SELECT "P2_[kW]" AS points
            FROM echarging
            WHERE "P2_[kW]" !=0
        UNION ALL
            SELECT "P3_[kW]" AS points
            FROM echarging
            WHERE "P3_[kW]" !=0
        UNION ALL
            SELECT "P4_[kW]" AS points
            FROM echarging
            WHERE "P4_[kW]" !=0)
                     
SELECT MAX(CAST(points AS INT)) AS max_power
FROM power''')

,max_power
0,360


**8. What is the distribution of charging points by power (in percentage)**

In [63]:
run('''
--concatenate all points
WITH power AS(
            SELECT "P1_[kW]" AS points
            FROM echarging
            WHERE "P1_[kW]" !=0
        UNION ALL
            SELECT "P2_[kW]" AS points
            FROM echarging
            WHERE "P2_[kW]" !=0
        UNION ALL
            SELECT "P3_[kW]" AS points
            FROM echarging
            WHERE "P3_[kW]" !=0
        UNION ALL
            SELECT "P4_[kW]" AS points
            FROM echarging
            WHERE "P4_[kW]" !=0),
-- select common points

    common as(
        SELECT points as Common_points,
               cast(COUNT(points) as REAL) AS Number
        FROM power
        GROUP BY points
        ORDER BY Number DESC
        LIMIT 10),
-- select other points

    other as(
        SELECT "Other KW" as other_points,
            COUNT(CAST(points AS INT)) -  (SELECT SUM(Number) FROM common) AS Number
        FROM power
    ),
 --concatenate common points and other points
 
    common_other as(
        SELECT * FROM common
            UNION ALL
        SELECT * FROM other
    
    )   
 --calculate percentages of common points including other
 
SELECT Common_points,
    round((Number/((SELECT COUNT(points) FROM power))*100),2) AS percentage
FROM common_other
''')
        

,Common_points,percentage
0,22.0,67.51
1,11.0,15.06
2,50.0,3.54
3,150.0,3.52
4,300.0,3.36
5,43.0,1.69
6,3.7,0.77
7,350.0,0.67
8,75.0,0.62
9,250.0,0.49


In [60]:
run('''
--concatenate all points
WITH charging_points AS(
            SELECT "P1_[kW]" AS points
            FROM echarging
            WHERE "P1_[kW]" !=0
        UNION ALL
            SELECT "P2_[kW]" AS points
            FROM echarging
            WHERE "P2_[kW]" !=0
        UNION ALL
            SELECT "P3_[kW]" AS points
            FROM echarging
            WHERE "P3_[kW]" !=0
        UNION ALL
            SELECT "P4_[kW]" AS points
            FROM echarging
            WHERE "P4_[kW]" !=0),
-- select common points

    common as(
        SELECT points as Common_points,
               cast(COUNT(points) as REAL) AS Number
        FROM charging_points
        GROUP BY points
        ORDER BY Number DESC
        LIMIT 10),
-- select other points

    other as(
        SELECT "Other KW" as other_points,
            COUNT(CAST(points AS INT)) -      
        (SELECT SUM(Number) 
                   FROM common) AS Number
        FROM charging_points
    ),
 --concatenate common points and other points
 
    common_other as(
        SELECT * FROM common
            UNION ALL
        SELECT * FROM other
    
    )   
 --calculate percentages of common points including other
 
        SELECT Common_points,
            round(
            (Number/((SELECT COUNT(points) FROM charging_points))*100)
            ,2) AS percentage
        FROM common_other
        ''')
        

,Common_points,percentage
0,22.0,67.51
1,11.0,15.06
2,50.0,3.54
3,150.0,3.52
4,300.0,3.36
5,43.0,1.69
6,3.7,0.77
7,350.0,0.67
8,75.0,0.62
9,250.0,0.49


**9. How many stations have with ultra-rapid charging points**

In [31]:
run('''
with a as(SELECT "P1_[kW]" AS points
            FROM echarging
            WHERE "P1_[kW]" !=0
        UNION ALL
            SELECT "P2_[kW]" AS points
            FROM echarging
            WHERE "P2_[kW]" !=0
        UNION ALL
            SELECT "P3_[kW]" AS points
            FROM echarging
            WHERE "P3_[kW]" !=0
        UNION ALL
            SELECT "P4_[kW]" AS points
            FROM echarging
            WHERE "P4_[kW]" !=0)
 --stations with any pointgreater than 300           
SELECT count(points) as ultra_fast_stations
FROM a
where points>=300
''')

,ultra_fast_stations
0,2975


**10. How many stations have ultra-rapid charging points per federal state**

In [32]:
run('''

WITH a AS(SELECT "P1_[kW]" AS points, Federal_State
            FROM echarging
            WHERE "P1_[kW]" !=0
        UNION ALL
            SELECT "P2_[kW]" AS points, Federal_State
            FROM echarging
            WHERE "P2_[kW]" !=0
        UNION ALL
            SELECT "P3_[kW]" AS points, Federal_State
            FROM echarging
            WHERE "P3_[kW]" !=0
        UNION ALL
            SELECT "P4_[kW]" AS points, Federal_State
            FROM echarging
            WHERE "P4_[kW]" !=0)
 --stations with any point greater than 300           
SELECT Federal_State,
            count(points) as Ultra_fast_stations
FROM a
WHERE points >= 300
GROUP BY Federal_State
ORDER BY Ultra_fast_stations DESC
''')

,Federal_State,Ultra_fast_stations
0,Bayern,705
1,Nordrhein-Westfalen,508
2,Baden-Württemberg,411
3,Niedersachsen,364
4,Hessen,234
5,Rheinland-Pfalz,140
6,Sachsen,120
7,Brandenburg,104
8,Sachsen-Anhalt,84
9,Schleswig-Holstein,73


**11. How many ultra-rapid charging points are installed per Year**

In [40]:
run('''
WITH a AS(SELECT "P1_[kW]" AS points, commissioning_date
            FROM echarging
            WHERE "P1_[kW]" !=0
        UNION ALL
            SELECT "P2_[kW]" AS points,commissioning_date
            FROM echarging
            WHERE "P2_[kW]" !=0
        UNION ALL
            SELECT "P3_[kW]" AS point,commissioning_date
            FROM echarging
            WHERE "P3_[kW]" !=0
        UNION ALL
            SELECT "P4_[kW]" AS points,commissioning_date
            FROM echarging
            WHERE "P4_[kW]" !=0)
--stations with any pointgreater than 300           
SELECT strftime('%Y', commissioning_date) year,
        count(points) as Ultra_rapid_stations
    FROM a
WHERE points>=300
GROUP BY year
ORDER BY year DESC
''')

,year,Ultra_rapid_stations
0,2022,1075
1,2021,980
2,2020,415
3,2019,389
4,2018,108
5,2017,2
6,2016,2
7,2012,2
8,2011,2


**12. What is the total number of ultra-rapid charging stations per city over the years**

In [44]:
run('''

WITH top_cities AS (SELECT Place,
                        sum(number_of_charging_points) Number_of_points
                            FROM echarging
                            GROUP BY Place
                            ORDER BY Number_of_points desc
                            LIMIT 10),
                            
cities AS(SELECT strftime('%Y',commissioning_date) AS Year,
                Place,
                    count(*) AS stations
                FROM echarging
                WHERE Place in(SELECT Place
                            FROM top_cities)
                            
                GROUP BY Year, Place
                ORDER BY Year ASC,stations ASC)
       
--running totals         
SELECT Year, 
            Place,
SUM(stations) OVER (PARTITION BY Place 
ORDER BY Year ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) cum_sum_stations
FROM cities
--WHERE Place='Berlin'
Group by Year,Place
LIMIT 15
''')

,Year,Place,cum_sum_stations
0,2009,Berlin,23
1,2010,Berlin,37
2,2011,Berlin,60
3,2012,Berlin,81
4,2014,Berlin,84
5,2015,Berlin,146
6,2016,Berlin,242
7,2017,Berlin,316
8,2018,Berlin,378
9,2019,Berlin,532


Here we get introduced to **window functions** in SQL. Breaking down the problem,for each city, we are doing a sum of the number of charging stations per year,to find the cumulative sum of stations up to the year we want to look at. We use a temporary table created by the query in exercise 2 and another temporary table which gives us the Year, Place and number of stations. We then perform the aggregation partitioning by Place and ordering by Year whereby  **Rows between unbounded preceding and current Row** means: for example if we look at 2022(latest year), we will get the sum of stations installed since 2022 including 2022 itself. This can be confirmed by the results we obtained in exercise 2.


**Exercise:** Add a column with the percentage increase per city per year in the number of stations compared with the previous year only. (If there is no previous year then return null)

**13. What are the Top 10 Operators and their respective numbers**

In [45]:
run('''
SELECT operator, count(operator) AS Number_of_operators
FROM echarging
GROUP BY operator
ORDER BY Number_of_operators DESC
LIMIT 10
''')

,operator,Number_of_operators
0,EnBW mobility+ AG und Co.KG,2058
1,E.ON Drive GmbH,1284
2,EWE Go GmbH,859
3,Westenergie Metering GmbH,737
4,Stromnetz Hamburg GmbH,670
5,SWM Versorgungs GmbH,598
6,Allego GmbH,587
7,ALDI SÜD,546
8,Audi AG,536
9,Berliner Stadtwerke EnergiePartner GmbH,510


**14. How many operators are there per Federal State**

In [64]:
run('''
SELECT Federal_State,
            operator,
                COUNT(operator) Number_of_operators
FROM echarging
GROUP BY Federal_State
ORDER BY Number_of_operators DESC
''')

,Federal_State,operator,Number_of_operators
0,Bayern,ALDI SÜD,7453
1,Nordrhein-Westfalen,Zahnärzte am Jutequartier,6837
2,Baden-Württemberg,EnBW mobility+ AG und Co.KG,6534
3,Niedersachsen,Auto Schrader GmbH,4041
4,Hessen,Ladegrün! eG,2944
5,Sachsen,EDEKA König,1484
6,Schleswig-Holstein,Stadtwerke Heide GmbH,1447
7,Rheinland-Pfalz,Allego GmbH,1415
8,Berlin,Verleyen,1080
9,Hamburg,Clipper Boardinghouse GmbH & Co. KG,990


**15. What are the most common type of connectors**

In [37]:
run(''' 
WITH connectors AS(SELECT connector_Types1 Connector_Type
                    FROM echarging
                    UNION ALL
                    SELECT connector_Types2 Connector_Type
                    FROM echarging
                    UNION ALL
                    SELECT connector_Types3 Connector_Type
                    FROM echarging
                    UNION ALL
                    SELECT connector_Types4 Connector_Type
                    FROM echarging
                    )
                    
SELECT Connector_Type, COUNT(Connector_Type) AS number_of_connectors
FROM connectors
WHERE Connector_type != 0
GROUP BY Connector_Type
ORDER BY number_of_connectors DESC
LIMIT 10
''')

,Connector_Type,number_of_connectors
0,AC Steckdose Typ 2,49736
1,DC Kupplung Combo,7785
2,AC Kupplung Typ 2,4540
3,"AC Steckdose Typ 2, AC Schuko",4519
4,"DC Kupplung Combo, DC CHAdeMO",2788
5,"AC Steckdose Typ 2, AC Kupplung Typ 2",1216
6,"AC Kupplung Typ 2, DC CHAdeMO",232
7,"AC Kupplung Typ 2, DC Kupplung Combo",205
8,AC Schuko,154
9,"AC Steckdose Typ 2, DC Kupplung Combo, DC CHAdeMO",70


## 4. Conclusion and Recommendations

We walked through alot, cleaning the Germany charging infrastructure dataset,and preparing it for querying and answering questions. We also set up SQLITE to enable us to run queries on our workbook. In practice,the data would be in data warehouses or databases but it is also possible to connect to it using the respective connectors depending on the database and so this method of working with the data.
With SQL we aren’t able to visualise the data. This is where libraries such as matplotlib or seaborn come in handy. 

For stakeholders in an organisation setting up dashboards is a  common practice. I have built a Tableau dashboard for this dataset which can be found [here](https://public.tableau.com/app/profile/samuel.konzi/viz/AnalysisofE-ChargingpointsinGermany/overview).

Feel free to dig with the data and answer the suggested questions I provided or even formulate and answer your own questions. Also if you have an optimal query option for any of the solutions let me know in the comments.
